## @Tool装饰器

In [2]:
from langchain.tools import tool

from chapter02.ModelInvocation import messages


@tool
def add_number(a: int, b: int) -> int:
    """两个整数相加"""
    return a + b

print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a": 10, "b": 20})
print(res)

name = add_number
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
description = 两个整数相加
return_direct = False
30


In [3]:
from langchain.tools import tool

@tool(name_or_callable="add_two_number", description="add two numbers to a result", return_direct=True)
def add_number(a: int, b: int) -> int:
    """整数之和"""
    return a + b

print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a": 10, "b": 20})
print(res)

name = add_two_number
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
description = add two numbers to a result
return_direct = True
30


In [9]:
from langchain.tools import tool
from pydantic import BaseModel, Field

class FieldInfo(BaseModel):
    a: int = Field(description="第一个参数")
    b: int = Field(description="第二个参数")

@tool(name_or_callable="add_two_number", description="add two numbers to a result", args_schema=FieldInfo, return_direct=True)
def add_number(a: int, b: int) -> int:
    """整数相加"""
    return a + b

print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a": 10, "b": 20})
print(res)

name = add_two_number
args = {'a': {'description': '第一个参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第二个参数', 'title': 'B', 'type': 'integer'}}
description = add two numbers to a result
return_direct = True
30


## StructuredTool的from_function()

In [10]:
from langchain_core.tools import StructuredTool

def search_function(query: str):
    return "Langchain"

search1 = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events"
)

print(f"name = {search1.name}")
print(f"description = {search1.description}")
print(f"args = {search1.args}")

search1.invoke("hello")

name = Search
description = useful for when you need to answer questions about current events
args = {'query': {'title': 'Query', 'type': 'string'}}


'Langchain'

In [11]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class FieldInfo(BaseModel):
    query: str = Field(description="检索的关键词")

def search_function(query: str):
    return "Langchain"

search1 = StructuredTool.from_function(
    func=search_function,
    name="Search",
    args_schema=FieldInfo,
    description="useful for when you need to answer questions about current events",
    return_direct=True
)

print(f"name = {search1.name}")
print(f"description = {search1.description}")
print(f"args = {search1.args}")
print(f"return_direct = {search1.return_direct}")

search1.invoke("hello")

name = Search
description = useful for when you need to answer questions about current events
args = {'query': {'description': '检索的关键词', 'title': 'Query', 'type': 'string'}}
return_direct = True


'Langchain'

## 通过大模型调用工具

In [18]:
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 定义工具
tools = [MoveFileTool()]
# 工具转成openai函数，后续可将函数进行模型调用
functions = [convert_to_openai_function(t) for t in tools]
# 提供消息
messages = [HumanMessage(content="我是Windows11系统，我的文件目录是E:\\a.txt,将文件a移动到C:\\Users\\admin\\Desktop\\a.txt")]
# 模型使用函数
response = chat_model.invoke(
    input=messages,
    functions=functions,
)

## 用于对比
# response = chat_model.invoke(
#     input=[HumanMessage(content="今天北京的天气怎么样？")],
#     functions=functions
# )
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"E:\\\\a.txt","destination_path":"C:\\\\Users\\\\admin\\\\Desktop\\\\a.txt"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 97, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CQm3goj62Ff6Neqy30De0qNiavpc0', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None} id='run--15e5423e-0fbf-4e46-896f-6f9071ac4f91-0' usage_metadata={'input_tokens': 97, 'output_tokens': 33, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:

# 检查是否需要调用工具
import json

if "function_call" in response.additional_kwargs:
    tool_name = response.additional_kwargs["function_call"]["name"]
    tool_args = json.loads((response.additional_kwargs["function_call"]["arguments"]))
    print(f"调用工具:{tool_name}，参数:{tool_args}")

else :
    print("模型回复：", response.content)


# 实际执行工具
if "move_file" in response.additional_kwargs["function_call"]["name"]:
    tool = MoveFileTool()
    result = tool.run(tool_args) # 执行工具
    print(result)

调用工具:move_file，参数:{'source_path': 'E:\\a.txt', 'destination_path': 'C:\\Users\\admin\\Desktop\\a.txt'}
File moved successfully from E:\a.txt to C:\Users\admin\Desktop\a.txt.
